In [1]:
# imports :
import pandas as pd

import numpy as np

import matplotlib.pyplot as plt

import tensorflow as tf

import tqdm # for the progress line

import glob # for the paths

from sklearn import model_selection

2024-08-09 20:54:22.973420: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-09 20:54:22.974903: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-09 20:54:23.004283: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-09 20:54:23.005362: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-09 20:54:23.527838: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
! pip install kaggle

Defaulting to user installation because normal site-packages is not writeable


In [3]:
# if we wnat to read dataset from kaggle directly , first put file kaggle.json
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/home/abdalrahman/.kaggle’: File exists


In [4]:
!kaggle datasets download -d jessicali9530/celeba-dataset

Dataset URL: https://www.kaggle.com/datasets/jessicali9530/celeba-dataset
License(s): other
100%|██████████████████████████████████████| 1.33G/1.33G [20:32<00:00, 2.05MB/s]
100%|██████████████████████████████████████| 1.33G/1.33G [20:32<00:00, 1.16MB/s]


In [5]:
!unzip /content/celeba-dataset.zip

unzip:  cannot find or open /content/celeba-dataset.zip, /content/celeba-dataset.zip.zip or /content/celeba-dataset.zip.ZIP.


In [6]:
def make_pair(folder_path):

  files_list=glob.glob(folder_path+'/*.jpg') # to pahs of the images

  file_list1 = []
  file_list2 = []

  target_list=[]

  for path in tqdm.tqdm(files_list):

      file_list1.append(path)
      file_list2.append(path)

      target_list.append(1)

      while True:
         n=np.random.randint(low=0 ,high=len(files_list))

         if files_list[n] != path:

            file_list1.append(path)
            file_list2.append(files_list[n])
            target_list.append(0)
            break
  data_dic={
      "image_1":file_list1,
      "image_2":file_list2,
      "label":target_list
      }

  dataset=pd.DataFrame(data=data_dic)


  return dataset

In [7]:
df=make_pair('/content/img_align_celeba/img_align_celeba').iloc[:10000]
df

0it [00:00, ?it/s]


,image_1,image_2,label


In [9]:
train,test=model_selection.train_test_split(df,test_size=0.1,random_state=42,stratify=df["label"])
train.shape,test.shape

ValueError: With n_samples=0, test_size=0.1 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:
def read_image(x,y):
    x1,x2=x

    x1=tf.io.read_file(x1)
    x1=tf.image.decode_jpeg(x1,channels=3)
    x1=tf.image.convert_image_dtype(x1,tf.float32)
    x1=tf.image.resize(x1,(244,244))


    x2=tf.io.read_file(x2)
    x2=tf.image.decode_jpeg(x2,channels=3)
    x2=tf.image.convert_image_dtype(x2,tf.float32)
    x2=tf.image.resize(x2,(244,244))




    return (x1,x2),y


In [ ]:
def train_preprocessing(x,y):

    x1,x2=x

    x1=tf.image.random_brightness(x1,max_delta=32.0/255.0)
    x1=tf.image.random_flip_left_right(x1)
    x1=tf.image.random_saturation(x1,lower=0.5,upper=1.5)


    x2=tf.image.random_brightness(x2,max_delta=32.0/255.0)
    x2=tf.image.random_flip_left_right(x2)
    x2=tf.image.random_flip_up_down(x2)
    x2=tf.image.random_saturation(x2,lower=0.5,upper=1.5)
    return (x1,x2),y

In [ ]:
def dataset_creater(x1,x2,y,batch_size=32,training=False):
    data=tf.data.Dataset.from_tensor_slices(((x1,x2),y))

    data=data.shuffle(1028) #  mean to take every 1028 elemens at once

    data=data.map(read_image,num_parallel_calls=tf.data.experimental.AUTOTUNE)

    if training:

        data=data.map(train_preprocessing,num_parallel_calls=tf.data.experimental.AUTOTUNE)


        data=data.batch(batch_size)

        data=data.prefetch(tf.data.experimental.AUTOTUNE)

        return data

In [ ]:
train_dataset=dataset_creater(train.image_1,train.image_2,train.label,training=True)
test_dataset=dataset_creater(test.image_1,test.image_2,test.label)

In [ ]:
n=30
f={"size":30,"color":"red"}
plt.figure(figsize=(50,5))

for x,y in train_dataset.take(1):

    x1,x2=x

    for i in range(n):

        plt.subplot(2,n,i+1)
        plt.title(f'{y[i]}',fontdict=f)
        plt.imshow(tf.squeeze(x1[i]))
        plt.gray()
        plt.xticks([])
        plt.yticks([])

        plt.subplot(2,n,n+i+1)
        plt.imshow(tf.squeeze(x2[i]))
        plt.gray()
        plt.xticks([])
        plt.yticks([])


In [ ]:
cnn_model=tf.keras.applications.densenet.DenseNet121(
    include_top=False,
    weights='imagenet',
    input_shape=(244,244,3)

)
cnn_model.summary()

In [ ]:
cnn_model.trainable=False
cnn_model.summary()

In [ ]:
class SiameseModel(tf.keras.Model):
  def __init__(self, cnn_model):
      super(SiameseModel, self).__init__()

      self.cnn_model = cnn_model
      self.d1 = tf.keras.layers.Dense(512, use_bias=False)
      self.bn1 = tf.keras.layers.BatchNormalization()
      self.d2 = tf.keras.layers.Dense(128, use_bias=False)
      self.bn2 = tf.keras.layers.BatchNormalization()
      self.out = tf.keras.layers.Dense(units=1, activation=tf.nn.sigmoid)

  def call(self, inputs):
    x1, x2 = inputs
    x1 = self.cnn_model(x1)
    x1 = tf.keras.layers.GlobalAveragePooling2D()(x1)
    x2 = self.cnn_model(x2)
    x2 = tf.keras.layers.GlobalAveragePooling2D()(x2)

    x = tf.keras.layers.concatenate([x1, x2])
    x = self.d1(x)
    x = self.bn1(x)
    x = tf.nn.leaky_relu(x)
    x = self.d2(x)
    x = self.bn2(x)
    x = tf.nn.leaky_relu(x)
    output = self.out(x)
    return output

  def build_model(self, dim=[244,244,3]):
    x = (tf.keras.layers.Input(dim), tf.keras.layers.Input(dim))
    return tf.keras.Model(inputs=[x], outputs=[self.call(x)])


model = SiameseModel(cnn_model)
model.build_model(dim=[244,244,3]).summary()

In [ ]:
tf.keras.utils.plot_model(model.build_model(dim=[244,244,3]))

In [ ]:
model.compile('adam',
              loss='binary_crossentropy',
              metrics=['acc'])

In [ ]:
model.fit(train_dataset,batch_size=32,epochs=6,validation_data=test_dataset)

In [ ]:
df=pd.DataFrame(model.history.history)
df

In [ ]:
df.plot()